<a href="https://colab.research.google.com/github/indhu68/Intro_to_DL_Project/blob/main/RTML3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch_geometric.nn as pyg_nn
import matplotlib.pyplot as plt
import torch_geometric.data as geom_data

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# Load data
data = pd.read_csv('/content/drive/My Drive/Kasungu_Telemetry_Pts_Oct23.csv', parse_dates=['Time.Stamp'])
data

Mounted at /content/drive


,Index,Tag,Type,Latitude,Longitude,Time.Stamp,DOP,Speed,Battery,Movement,Alarm,RSSI,Coverage,Retries,SW_Ver_,Time_Stamp,Log_Interv,Temperatur,Accelerome,SW.Ver.
0,10000,5748,IR-SAT Tag,-13.027828,33.133265,2022-07-14 11:08:00,10 meters,3 km/h,3.68 Volt,Y,N,5,N,4,66,7/14/2022,Every 30 minutes,41,"[1033,5505,15905]",66
1,10001,5744,IR-SAT Tag,-13.224110,33.157962,2022-07-14 11:10:00,<10 meters,0 km/h,3.65 Volt,Y,N,4,Y,0,66,7/14/2022,Every 30 minutes,41,"[1222,-960,15876]",66
2,10002,5740,IR-SAT Tag,-13.050495,33.152538,2022-07-14 11:23:00,10 meters,0 km/h,3.68 Volt,Y,N,5,N,2,66,7/14/2022,Every 30 minutes,39,"[-1007,203,15311]",66
3,10005,5746,IR-SAT Tag,-13.061195,33.182005,2022-07-14 11:24:00,10 meters,2 km/h,3.66 Volt,Y,N,3,N,2,66,7/14/2022,Every 30 minutes,41,"[2940,984,14900]",66
4,10006,5745,IR-SAT Tag,-13.007968,33.104750,2022-07-14 11:24:00,10 meters,0 km/h,3.67 Volt,Y,N,5,N,2,66,7/14/2022,Every 30 minutes,40,"[721,-1718,15198]",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383055,9995,5751,IR-SAT Tag,-12.732593,33.183252,2023-08-16 00:04:00,10 meters,0 km/h,3.66 Volt,Y,N,2,N,3,66,8/16/2023,Every 1 hour,25,"[-441,284,15724]",66
383056,9996,5750,IR-SAT Tag,-12.730062,33.184862,2023-08-16 00:06:00,10 meters,2 km/h,3.64 Volt,Y,N,1,Y,0,66,8/16/2023,Every 1 hour,26,"[-1446,818,15792]",66
383057,9997,5757,IR-SAT Tag,-12.719173,33.082990,2023-08-16 00:12:00,10 meters,0 km/h,3.68 Volt,Y,N,4,Y,0,66,8/16/2023,Every 1 hour,23,"[-14394,1689,6386]",66
383058,9998,5743,IR-SAT Tag,-13.035063,33.221140,2023-08-16 00:17:00,10 meters,0 km/h,3.69 Volt,Y,N,5,Y,0,66,8/16/2023,Every 1 hour,22,"[2096,3363,15104]",66


In [8]:
data = data[["Index","Tag", "Latitude", "Longitude", "Time.Stamp"]]
data['Time.Stamp'] = pd.to_datetime(data['Time.Stamp'])
data.sort_values(by=['Tag', 'Time.Stamp'], inplace=True)


In [10]:
# Normalize latitude and longitude
scaler = MinMaxScaler()
data[['Latitude', 'Longitude']] = scaler.fit_transform(data[['Latitude', 'Longitude']])

In [12]:
# Function to create sequences
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps + 1  # Ensure there is enough data for this sequence
        if end_ix > len(data):
            break  # Break if end_ix exceeds the data length
        seq_x, seq_y = data.iloc[i:end_ix-1].to_numpy(), data.iloc[end_ix-1].to_numpy()[:2]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


# Generate sequences for each elephant
elephant_sequences = []
grouped = data.groupby('Tag')

for _, group in grouped:
    sequences = create_sequences(group[['Latitude', 'Longitude']], n_steps=5)
    elephant_sequences.append(sequences)

# Combine sequences into training and test data
X = np.concatenate([seq[0] for seq in elephant_sequences])
y = np.concatenate([seq[1] for seq in elephant_sequences])

In [13]:


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders for training and testing
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=True)


In [15]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=2, hidden_size=128, num_layers=2, batch_first=True)
        self.fc = nn.Linear(128, 2)  # Output layer to predict next latitude and longitude

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # Output from the last LSTM cell

# Create model, loss function, and optimizer
model = LSTMModel()
model = model.to('cuda')  # Move to GPU
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to('cuda')  # Move to GPU
            y_batch = y_batch.to('cuda')  # Move to GPU
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

# Train the LSTM model
train(model, train_loader, criterion, optimizer, num_epochs=10)


Epoch 1, Loss: 0.0010931561631219245
Epoch 2, Loss: 5.618043954141315e-05
Epoch 3, Loss: 1.92183385417406e-05
Epoch 4, Loss: 1.5162313925390823e-05
Epoch 5, Loss: 1.3752350055794732e-05
Epoch 6, Loss: 1.3013146247586599e-05
Epoch 7, Loss: 1.2165854943311861e-05
Epoch 8, Loss: 1.2100082281544712e-05
Epoch 9, Loss: 1.1827341565198135e-05
Epoch 10, Loss: 1.166167927052116e-05


In [ ]:
# Function to build a graph based on proximity or interaction
def build_graph(data, distance_threshold=0.1):
    nodes = data['Tag'].unique()  # Nodes are elephants
    node_indices = {elephant_id: idx for idx, elephant_id in enumerate(nodes)}

    # Create edges based on some interaction criterion, e.g., proximity
    edge_list = []
    for i, row_i in data.iterrows():
        for j, row_j in data.iterrows():
            if i != j:
                distance = np.linalg.norm([row_i['Latitude'] - row_j['Latitude'], row_i['Longitude'] - row_j['Longitude']])
                if distance < distance_threshold:
                    edge_list.append([node_indices[row_i['Tag']], node_indices[row_j['Tag']]])

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    x = torch.eye(len(nodes))  # Node features (identity matrix for simplicity)

    return geom_data.Data(x=x, edge_index=edge_index)

# Build the graph from the data
graph_data = build_graph(data, distance_threshold=0.1)

# Define GIN model
class GINModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = pyg_nn.GINConv(nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 16)
        ))
        self.conv2 = pyg_nn.GINConv(nn.Sequential(
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        ))

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return x

# Create the GIN model and optimizer
gin_model = GINModel().to('cuda')  # Move to GPU
gin_optimizer = torch.optim.Adam(gin_model.parameters(), lr=0.01)

# Function to train the GIN model
def train_gin(model, graph_data, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(graph_data.x.to('cuda'), graph_data.edge_index.to('cuda'))  # Move to GPU
        loss = torch.mean((output - graph_data.x.to('cuda')) ** 2)  # Example loss function
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Train the GIN model
train_gin(gin_model, graph_data, gin_optimizer, num_epochs=10)


In [ ]:
# Predicting with the LSTM model
model.eval()  # Set model to evaluation mode
predictions = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to('cuda')  # Move to GPU
        output = model(X_batch)
        predictions.append(output.cpu().numpy())  # Convert back to CPU for further processing

# Flatten predictions
predictions = np.concatenate(predictions, axis=0)

# Convert predictions and actual values back to original scale
predicted_coords = scaler.inverse_transform(predictions)
actual_coords = scaler.inverse_transform(y_test_tensor.cpu().numpy())

# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.plot(actual_coords[:, 0], actual_coords[:, 1], 'ro-', label='Actual Path')
plt.plot(predicted_coords[:, 0], predicted_coords[:, 1], 'bo-', label='Predicted Path')
plt.title('Actual vs Predicted Elephant Movement')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend()
plt.show()

# Visualize GIN embeddings to understand social patterns
gin_model.eval()  # Set GIN model to evaluation mode
with torch.no_grad():
    node_embeddings = gin_model(graph_data.x.to('cuda'), graph_data.edge_index.to('cuda'))

# Reduce embeddings to 2D for visualization
from sklearn.manifold import TSNE
embeddings_2d = TSNE(n_components=2).fit_transform(node_embeddings.cpu().numpy())

plt.figure(figsize=(10, 6))
for i in range(embeddings_2d.shape[0]):
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], label=f'Elephant {i}')
plt.title('Elephant Social Patterns (GIN Embeddings)')
